In [3]:
import sys
import os

project_path = 'c:/Users/Firew Ayele/Desktop/kifiya/telecom_analysis'
sys.path.append(project_path)

import pandas as pd
from Database.db_connection import fetch_data_from_postgres
from src.utils import missing_values_table, convert_bytes_to_megabytes

fetch data from postgres

In [4]:
df =fetch_data_from_postgres()
print(df)

           Bearer Id            Start  Start ms              End  End ms  \
0       1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1       1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2       1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3       1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4       1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   
...              ...              ...       ...              ...     ...   
149996  7.277826e+18   4/29/2019 7:28     451.0   4/30/2019 6:02   214.0   
149997  7.349883e+18   4/29/2019 7:28     483.0  4/30/2019 10:41   187.0   
149998  1.311448e+19   4/29/2019 7:28     283.0  4/30/2019 10:46   810.0   
149999  1.311448e+19   4/29/2019 7:28     696.0  4/30/2019 10:40   327.0   
150000           NaN             None       NaN             None     NaN   

        Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0       1823652.0  2.08

In [5]:
# Perform initial data analysis
missing_values_df = missing_values_table(df)
print("Missing Values in df:")
print(missing_values_df)

Your selected dataframe has 55 columns.
There are 41 columns that have missing values.
Missing Values in df:
                                          Missing Values  % of Total Values  \
Nb of sec with 37500B < Vol UL                    130254               86.8   
Nb of sec with 6250B < Vol UL < 37500B            111843               74.6   
Nb of sec with 125000B < Vol DL                    97538               65.0   
TCP UL Retrans. Vol (Bytes)                        96649               64.4   
Nb of sec with 31250B < Vol DL < 125000B           93586               62.4   
Nb of sec with 1250B < Vol UL < 6250B              92894               61.9   
Nb of sec with 6250B < Vol DL < 31250B             88317               58.9   
TCP DL Retrans. Vol (Bytes)                        88146               58.8   
HTTP UL (Bytes)                                    81810               54.5   
HTTP DL (Bytes)                                    81474               54.3   
Avg RTT DL (ms)       

In [6]:
df['HTTP DL (MB)'] = convert_bytes_to_megabytes(df, 'HTTP DL (Bytes)')

print(df.head())

      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube UL (Bytes)  Netflix DL (Bytes)  \
0  9.16456699548519E+015  ...           2501332.0           8198936.0   
1                L77566A  ...          19111729.0          18338413.0   

In [7]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Aggregate metrics per customer (MSISDN)
customer_engagement = df.groupby('MSISDN/Number').agg(
    session_frequency=('Bearer Id', 'count'),
    total_session_duration=('Dur. (ms)', 'sum'),
    total_DL=('Total DL (Bytes)', 'sum'),
    total_UL=('Total UL (Bytes)', 'sum')
)
customer_engagement['total_traffic'] = customer_engagement['total_DL'] + customer_engagement['total_UL']

# Top 10 customers per engagement metric
top_10_frequency = customer_engagement.nlargest(10, 'session_frequency')
top_10_duration = customer_engagement.nlargest(10, 'total_session_duration')
top_10_traffic = customer_engagement.nlargest(10, 'total_traffic')

# Top 10 Customers by Session Frequency
print(top_10_frequency)
# Top 10 Customers by Session Duration
print(top_10_duration)
# Top 10 Customers by Session Total Traffic
print(top_10_traffic)

               session_frequency  total_session_duration      total_DL  \
MSISDN/Number                                                            
3.362632e+10                  18               8791927.0  7.301517e+09   
3.361489e+10                  17               9966898.0  8.156743e+09   
3.362578e+10                  17              18553754.0  7.770043e+09   
3.365973e+10                  16               4035428.0  7.081602e+09   
3.367588e+10                  15               4865947.0  7.309542e+09   
3.376054e+10                  15               9279434.0  7.811295e+09   
3.366716e+10                  13               8744914.0  5.052068e+09   
3.360313e+10                  12               6287761.0  4.540607e+09   
3.360452e+10                  12               5207990.0  5.096079e+09   
3.362708e+10                  12               4703516.0  5.309479e+09   

                  total_UL  total_traffic  
MSISDN/Number                              
3.362632e+10   66965072

In [8]:

scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(customer_engagement[['session_frequency', 'total_session_duration', 'total_traffic']])
print(normalized_data)

[[0.05555556 0.00590825 0.09593143]
 [0.05555556 0.00938651 0.01402598]
 [0.05555556 0.0068922  0.06385101]
 ...
 [0.05555556 0.04692194 0.02256615]
 [0.05555556 0.01325784 0.06388747]
 [0.05555556 0.04651534 0.0120169 ]]


In [9]:
engagement_data = customer_engagement[['session_frequency', 'total_session_duration', 'total_traffic']]
print(engagement_data)



               session_frequency  total_session_duration  total_traffic
MSISDN/Number                                                          
3.360100e+10                   1                116720.0   8.786906e+08
3.360100e+10                   1                181230.0   1.568596e+08
3.360100e+10                   1                134969.0   5.959665e+08
3.360101e+10                   1                 49878.0   4.223207e+08
3.360101e+10                   2                 37104.0   1.457411e+09
...                          ...                     ...            ...
3.379000e+10                   1                  8810.0   7.146416e+08
3.379000e+10                   1                140988.0   4.803073e+08
3.197021e+12                   1                877385.0   2.321240e+08
3.370000e+14                   1                253030.0   5.962878e+08
8.823971e+14                   1                869844.0   1.391536e+08

[106856 rows x 3 columns]


In [10]:
# Average tcp retransmission
average_tcp_retransmission = df.groupby('MSISDN/Number')[['TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)']].mean().sum(axis=1)
# Average_rtt
average_rtt = df.groupby('MSISDN/Number')[['Avg RTT DL (ms)', 'Avg RTT UL (ms)']].mean().sum(axis=1)

# Most frequent handset type
most_frequent_handset_type = df.groupby('MSISDN/Number')['Handset Type'].agg(lambda x: x.value_counts().index[0])

# Average throughput
average_throughput = df.groupby('MSISDN/Number')[['Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)']].mean().sum(axis=1)

# Aggregate all information into a DataFrame
customer_engagement_metrics = pd.DataFrame({
    'MSISDN/Number': average_tcp_retransmission.index,
    'Average TCP Retransmission': average_tcp_retransmission.values,
    'Average RTT': average_rtt.values,
    'Handset Type': most_frequent_handset_type.values,
    'Average Throughput': average_throughput.values
})

# Display the aggregated information
print(customer_engagement_metrics.head())

   MSISDN/Number  Average TCP Retransmission  Average RTT  \
0   3.360100e+10                         0.0         46.0   
1   3.360100e+10                         0.0         31.0   
2   3.360100e+10                         0.0          0.0   
3   3.360101e+10                      1066.0         84.0   
4   3.360101e+10                   9370832.0         59.5   

                     Handset Type  Average Throughput  
0  Huawei P20 Lite Huawei Nova 3E                76.0  
1          Apple iPhone 7 (A1778)                99.0  
2                       undefined                97.0  
3         Apple iPhone 5S (A1457)               248.0  
4         Apple iPhone Se (A1723)             28422.0  


In [11]:
print(engagement_data.columns)

Index(['session_frequency', 'total_session_duration', 'total_traffic'], dtype='object')


In [12]:
# Calculate average TCP retransmission
df['Average TCP Retransmission'] = (df['TCP DL Retrans. Vol (Bytes)'] + df['TCP UL Retrans. Vol (Bytes)']) / 2

# Calculate average RTT
df['Average RTT'] = (df['Avg RTT DL (ms)'] + df['Avg RTT UL (ms)']) / 2

# Calculate average throughput
df['Average Throughput'] = (df['Avg Bearer TP DL (kbps)'] + df['Avg Bearer TP UL (kbps)']) / 2


In [13]:

import pandas as pd
from sklearn.cluster import KMeans
from scipy.spatial import distance
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import StandardScaler


# engagement_data = customer_engagement[['session_frequency', 'total_session_duration', 'total_traffic']]

In [14]:
# Perform clustering analysis for engagement
engagement_data = customer_engagement[['session_frequency', 'total_session_duration', 'total_traffic']]
engagement_kmeans = KMeans(n_clusters=3)  # Assuming 3 clusters
engagement_kmeans.fit(engagement_data)

# Create a DataFrame for engagement labels
engagement_labels_df = pd.DataFrame({'Engagement Cluster': engagement_kmeans.labels_})


In [15]:
# Concatenate the labels DataFrame with the original DataFrame
df = pd.concat([df, engagement_labels_df], axis=1)


In [16]:
print(df)

           Bearer Id            Start  Start ms              End  End ms  \
0       1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1       1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2       1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3       1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4       1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   
...              ...              ...       ...              ...     ...   
149996  7.277826e+18   4/29/2019 7:28     451.0   4/30/2019 6:02   214.0   
149997  7.349883e+18   4/29/2019 7:28     483.0  4/30/2019 10:41   187.0   
149998  1.311448e+19   4/29/2019 7:28     283.0  4/30/2019 10:46   810.0   
149999  1.311448e+19   4/29/2019 7:28     696.0  4/30/2019 10:40   327.0   
150000           NaN             None       NaN             None     NaN   

        Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0       1823652.0  2.08

In [17]:
# Extract centroids
less_engaged_centroid = engagement_kmeans.cluster_centers_[0]  # Centroid of less engaged cluster


In [60]:
# Calculate average TCP retransmission
df['Average TCP Retransmission'] = (df['TCP DL Retrans. Vol (Bytes)'] + df['TCP UL Retrans. Vol (Bytes)']) / 2

# Calculate average RTT
df['Average RTT'] = (df['Avg RTT DL (ms)'] + df['Avg RTT UL (ms)']) / 2

# Calculate average throughput
df['Average Throughput'] = (df['Avg Bearer TP DL (kbps)'] + df['Avg Bearer TP UL (kbps)']) / 2


In [71]:
# Perform clustering analysis for experience
experience_data = customer_engagement_metrics[['Average TCP Retransmission', 'Average RTT', 'Average Throughput']]
experience_kmeans = KMeans(n_clusters=3)  # Assuming 3 clusters
# Drop rows with missing values
# experience_data.dropna(inplace=True)

# Reset index of experience_data to ensure alignment with df
# experience_data.reset_index(drop=True, inplace=True)

# Fit KMeans clustering algorithm
experience_kmeans.fit(experience_data)

# Assign cluster labels to a new DataFrame
experience_labels_df = pd.DataFrame({'Experience Cluster': experience_kmeans.labels_})

# Concatenate the labels DataFrame with the original DataFrame
df = pd.concat([df, experience_labels_df], axis=1)

In [72]:
# Extract centroids
worst_experience_centroid = experience_kmeans.cluster_centers_[0]  # Centroid of worst experience cluster


In [73]:
print(worst_experience_centroid)

[3.31753952e+06 1.12250629e+02 1.29872096e+04]


In [74]:
import numpy as np

# Define function to calculate Euclidean distance
def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2)**2))

# Calculate engagement score for each user
customer_engagement['Engagement Score'] = customer_engagement.apply(
    lambda row: euclidean_distance(np.array([row['session_frequency'], row['total_session_duration'], row['total_traffic']]), less_engaged_centroid),
    axis=1
)

df['Experience Score'] = df.apply(lambda row: distance.euclidean(row[['Average TCP Retransmission', 'Average RTT', 'Average Throughput']], worst_experience_centroid), axis=1)

# Calculate experience score for each user
customer_engagement['Experience Score'] = experience_data.apply(
    lambda row: euclidean_distance(np.array([row['Average TCP Retransmission'], row['Average RTT'], row['Average Throughput']]), worst_experience_centroid),
    axis=1
)




In [75]:
print(customer_engagement[['Engagement Score','Experience Score']])

               Engagement Score  Experience Score
MSISDN/Number                                    
3.360100e+10       5.458308e+08               NaN
3.360100e+10       1.760002e+08               NaN
3.360100e+10       2.631067e+08               NaN
3.360101e+10       8.946090e+07               NaN
3.360101e+10       1.124551e+09               NaN
...                         ...               ...
3.379000e+10       3.817818e+08               NaN
3.379000e+10       1.474475e+08               NaN
3.197021e+12       1.007388e+08               NaN
3.370000e+14       2.634280e+08               NaN
8.823971e+14       1.937077e+08               NaN

[106856 rows x 2 columns]


In [76]:
# Compute Euclidean distances for Experience Score
# df['Experience Score'] = df.apply(lambda row: distance.euclidean(row[['Average TCP Retransmission', 'Average RTT', 'Average Throughput']], worst_experience_centroid), axis=1)


In [77]:
# Display user data with scores
# print(df[['MSISDN/Number', 'Engagement Score', 'Experience Score']])

In [78]:
print(customer_engagement)

               session_frequency  total_session_duration      total_DL  \
MSISDN/Number                                                            
3.360100e+10                   1                116720.0  8.426375e+08   
3.360100e+10                   1                181230.0  1.207552e+08   
3.360100e+10                   1                134969.0  5.566597e+08   
3.360101e+10                   1                 49878.0  4.019932e+08   
3.360101e+10                   2                 37104.0  1.363130e+09   
...                          ...                     ...           ...   
3.379000e+10                   1                  8810.0  6.879252e+08   
3.379000e+10                   1                140988.0  4.445751e+08   
3.197021e+12                   1                877385.0  1.948281e+08   
3.370000e+14                   1                253030.0  5.396350e+08   
8.823971e+14                   1                869844.0  7.869760e+07   

                 total_UL  total_traf

In [79]:
# engagement_data = customer_engagement[['session_frequency', 'total_session_duration', 'total_traffic']]  # engagement features

# experience_data = df[['Average TCP Retransmission', 'Average RTT', 'Average Throughput']]  # experience features


In [80]:
# # KMeans models from previous tasks (Task 2 for engagement, Task 3 for experience)
# from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(strategy='mean')

# # Impute the missing values in engagement and experience data
# engagement_data_imputed = imputer.fit_transform(engagement_data)
# experience_data_imputed = imputer.fit_transform(experience_data)

# scaler = StandardScaler()
# engagement_data_scaled = scaler.fit_transform(engagement_data_imputed)

# experience_data_scaled = scaler.fit_transform(experience_data_imputed)

# # perform a k-means clustering (where k = 3)
# engagement_kmeans = KMeans(n_clusters=3, random_state=42)
# customer_engagement['Cluster'] = engagement_kmeans.fit_predict(engagement_data_scaled)



# experience_kmeans = KMeans(n_clusters=3, random_state=42)
# customer_engagement_metrics['Cluster'] = experience_kmeans.fit_predict(experience_data_scaled)

# # engagement_kmeans = KMeans(n_clusters=3, random_state=42).fit(engagement_data)

# # experience_kmeans = KMeans(n_clusters=3, random_state=42).fit(experience_data)
